In [1]:
!pip install thefuzz

In [2]:
import pandas as pd
import numpy as np
import re
import spacy
from thefuzz import process

In [3]:
# Load one of spacy's english NLP models
nlp = spacy.load("en_core_web_sm")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/10-K_report_ext.csv')
df

,Company,Year,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow from Operating Activities,Revenue Growth (%),Net Income Growth (%),Assets Growth (%),Liabilities Growth (%),Cash Flow Growth (%),Equity,Equity Growth (%),Equity to Asset Ratio,CFO to Net Income ratio
0,Microsoft,2020,143015.0,44281.0,NaN,NaN,60675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.370227
1,Microsoft,2021,168088.0,61271.0,333779.0,191791.0,76740,17.531727,38.368601,NaN,NaN,26.477132,141988.0,NaN,0.425395,1.252469
2,Microsoft,2022,198270.0,72738.0,364840.0,198298.0,89035,17.956071,18.715216,9.305858,3.392756,16.021631,166542.0,17.293011,0.456480,1.224051
3,Microsoft,2023,211915.0,72361.0,411976.0,205753.0,87582,6.882030,-0.518299,12.919636,3.759493,-1.631942,206223.0,23.826422,0.500570,1.210348
4,Microsoft,2024,245122.0,88136.0,512163.0,243686.0,118548,15.669962,21.800417,24.318650,18.436183,35.356580,268477.0,30.187709,0.524202,1.345058
5,Tesla,2019,NaN,NaN,NaN,NaN,2405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Tesla,2020,31540.0,721.0,52148.0,28418.0,5943,NaN,NaN,NaN,NaN,147.110187,23730.0,NaN,0.455051,8.242718
7,Tesla,2021,53820.0,5519.0,62131.0,30548.0,11497,70.640457,665.464632,19.143591,7.495249,93.454484,31583.0,33.093131,0.508329,2.083167
8,Tesla,2022,81460.0,12556.0,82338.0,36440.0,14724,51.356373,127.504983,32.523217,19.287678,28.068192,45898.0,45.325017,0.557434,1.172666
9,Tesla,2023,96770.0,14997.0,106618.0,43009.0,13256,18.794500,19.440905,29.488207,18.026894,-9.970117,63609.0,38.587738,0.596607,0.883910


In [8]:
figures = list(df.columns[2:])
figures

['Total Revenue',
 'Net Income',
 'Total Assets',
 'Total Liabilities',
 'Cash Flow from Operating Activities',
 'Revenue Growth (%)',
 'Net Income Growth (%)',
 'Assets Growth (%)',
 'Liabilities Growth (%)',
 'Cash Flow Growth (%)',
 'Equity',
 'Equity Growth (%)',
 'Equity to Asset Ratio',
 'CFO to Net Income ratio']

In [10]:
def extract_entities(user_query):
    """Uses an NLP model to extract company names (always capitalized), years, and financial figures from the user query."""

    # Apply the model to the user query
    doc = nlp(user_query)

    # Initialize the variables of interest
    company = None
    year = None
    figure = None

    # Extract company name
    for ent in doc.ents:
        if ent.label_ == "ORG":  # Organizations (companies), names always capitalized
            if ent.text in df["Company"].values:
                company = ent.text
                break

    # Extract year (4-digit number)
    for token in doc:
        if token.text.isdigit() and len(token.text) == 4:
            year = int(token.text)
            break

    # Extract financial figure using fuzzy matching
    metric, confidence = process.extractOne(user_query, figures)
    if confidence > 80:
        figure = metric

    # Extract a list of keywords in case the user wants to perform aggregation functions
    aggregation_keywords = {"mean": 'mean',
                            "average": 'mean',
                            "sum": 'sum',
                            "count": 'count',
                            "min": 'min',
                            "minimum": 'min',
                            "maximum": 'max',
                            "max": 'max',
                            "median": 'median',
                            "mode": 'mode',
                            "std": 'std',
                            "standard deviation": 'std',
                            "variance": 'var'}

    pattern = re.compile(r"\b(" + "|".join(aggregation_keywords.keys()) + r")\b", re.IGNORECASE)
    matches = pattern.findall(doc.text.lower())
    matches_dict = {}
    for key in matches:
        matches_dict[key] = aggregation_keywords[key]

    return company, year, figure, matches_dict

In [19]:
def simple_chatbot(df):
    """Handles the user queries."""
    print("Bot: Hello! I'm a chatbot specialized in financial queries. How can I assist you today?")

    user_query = input("You: ")

    # Call the function extract_entities to identify company, year and figure from the query
    company, year, figure, matches_dict = extract_entities(user_query)

    # Handle missing information
    if not company:
        company = extract_entities(input("Bot: Sorry, I couldn't identify the company in your query. Please specify it: ").capitalize())[0]
    if not figure:
        figure = extract_entities(input("Bot: Sorry, I couldn't identify the financial metric in your query. Please specify it: "))[2]
    if not year and not matches_dict:
        year = extract_entities(input("Bot: Sorry, I couldn't identify the year in your query. Please specify it: "))[1]

    # Retrieve the data points of interest
    point = df[(df["Company"] == company) & (df["Year"] == year)][figure]

    if not matches_dict:
        try:
            if np.isnan(point.values[0]):
                return "Bot: Sorry, the information you are looking for cannot be obtained from the dataset provided."
            elif not point.empty:
                if re.search(r"%", figure):
                    return "Bot: {}'s {} in {} was {}%.".format(company, re.sub(r"\s\(%\)", "", figure), year, round(point.values[0], 1))
                else:
                    return "Bot: {}'s {} in {} was {} million USD.".format(company, figure, year, point.values[0])
            else:
                return "Bot: Sorry, the information you are looking for cannot be obtained from the dataset provided."
        except IndexError:
            return "Bot: Sorry, the information you are looking for cannot be obtained from the dataset provided."
    else:
        for func in matches_dict.keys():
            result = df[(df['Company']==company)][figure].agg(matches_dict[func])
            if re.search(r"%", figure):
                print("Bot: The {} {} for {} is: {}%.".format(func, figure, company, round(result, 1)))
            else:
                print("Bot: The {} {} for {} is: {} million USD.".format(func, figure, company, round(result, 1)))

In [12]:
simple_chatbot(df) # figure in million USD

Bot: Hello! I'm a chatbot specialized in financial queries. How can I assist you today?
You: What were Microsoft's total assets in 2023?


"Bot: Microsoft's Total Assets in 2023 was 411976.0 million USD."

In [13]:
simple_chatbot(df) # retrieves NaN value

Bot: Hello! I'm a chatbot specialized in financial queries. How can I assist you today?
You: What were Microsoft's total assets in 2020?


'Bot: Sorry, the information you are looking for cannot be obtained from the dataset provided.'

In [14]:
simple_chatbot(df) # year not in the dataframe

Bot: Hello! I'm a chatbot specialized in financial queries. How can I assist you today?
You: What was Microsoft's equity in 1999?


'Bot: Sorry, the information you are looking for cannot be obtained from the dataset provided.'

In [15]:
simple_chatbot(df) # figure in percentage %

Bot: Hello! I'm a chatbot specialized in financial queries. How can I assist you today?
You: Hey dear chatbot! I would really like to know, just out of curiosity, what was tesla's revenue growth in 2022?
Bot: Sorry, I couldn't identify the company in your query. Please specify it: tesla


"Bot: Tesla's Revenue Growth in 2022 was 51.4%."

In [16]:
simple_chatbot(df) # company not in the data frame

Bot: Hello! I'm a chatbot specialized in financial queries. How can I assist you today?
You: Tell me the CFO to net income ratio of Costco.
Bot: Sorry, I couldn't identify the company in your query. Please specify it: Costco
Bot: Sorry, I couldn't identify the year in your query. Please specify it: 2024


'Bot: Sorry, the information you are looking for cannot be obtained from the dataset provided.'

In [20]:
simple_chatbot(df) # Mean value in million USD

Bot: Hello! I'm a chatbot specialized in financial queries. How can I assist you today?
You: What's the average equity for Apple?
Bot: The average Equity for Apple is: 58214.5 million USD.


In [25]:
simple_chatbot(df) # If the query is extended, the model is not able to differentiate Equity from Equity Growth (%)
                  # Here I'm asking for the average equity, and the bot gives information about equity growth (%), which is a different variable

Bot: Hello! I'm a chatbot specialized in financial queries. How can I assist you today?
You: What's the average equity for Apple for the last 10 years?
Bot: The average Equity Growth (%) for Apple is: -1.8%.


In [22]:
simple_chatbot(df) # Mean value in percentage %

Bot: Hello! I'm a chatbot specialized in financial queries. How can I assist you today?
You: Tell me the mean revenue growth for apple
Bot: Sorry, I couldn't identify the company in your query. Please specify it: Apple
Bot: The mean Revenue Growth (%) for Apple is: 10.1%.


In [24]:
simple_chatbot(df) # Another aggregation function

Bot: Hello! I'm a chatbot specialized in financial queries. How can I assist you today?
You: What was the minimum net income for Tesla?
Bot: The minimum Net Income for Tesla is: 721.0 million USD.


**Comments:**

1. This chatbot works better if the name of the company is capitalized in the query.

2. The chatbot presents an issue with fuzzy words, and is not able to differentiate, for example, equity from equity growth (%), when the query is extended and contains irrelevant information (This is probably due to the confidence level used when extracting figures with process.extractOne). Thus, the user queries should be as succinct as possible.

3. I coded this version of the chatbot so it could calculate several aggregation functions and not only the mean. But it does the calculations over all the years that are available in the data set.